In [ ]:
# Set up GRASS environment
import os
import csv
from IPython.display import Image
from helper import show_interactively, show
import grass.script as gs
# had to add shell = True to helper.py for this to work
from helper import initialize_GRASS_notebook
initialize_GRASS_notebook(binary='grass83',
                          grassdata="D:\\grassdata", location="FUTURES_zoning_prelim",
                          mapset="input")

In [ ]:
def get_pred(file_name):
  file = open(file_name)
  csvreader = csv.reader(file)
  pred = []
  # remove first three columns because they are always ID, Intercept, and devpressure
  pred = next(csvreader)[3:]
  file.close()
  return pred

In [ ]:
train_years = [2001, 2004, 2006, 2008, 2011]
validate_years = [2013, 2016, 2019, 2021]

# subregions
subregion = 'sa_counties'
dev_start = 'urban_2001'
dev_end = 'urban_2011'

discount_factor = [0.1]
compactness_mean = [0, 0.2, 0.4]
compactness_range = [0.1]

# number of CPUs
nprocs = 7

In [ ]:
def calibrate(potential, demand, calib, patches, rep):
  predictors = get_pred(potential)
  gs.run_command('r.futures.calib', development_start=dev_start, development_end=dev_end,
               subregions=subregion, patch_sizes=patches, patch_threshold=1800, flags='s',
               repeat=rep, calibration_results=calib, nprocs=nprocs,
               predictors=predictors,
               devpot_params=potential, development_pressure='devpressure_30_05_01_2011',
               n_dev_neighbourhood=37, development_pressure_approach='gravity', gamma=0.5, scaling_factor=0.1,
               demand=demand, discount_factor=discount_factor, compactness_mean=compactness_mean,
               compactness_range=compactness_range, num_neighbors=4, seed_search='probability', random_seed=1, overwrite=True)

calibrate depending on parameters

In [ ]:
#TODO do I need to re-calibrate for each variation of potential?

In [ ]:
# no zoning
calibrate('potential.csv', 'demand.csv', 'calib.csv', 'patches.csv', 50)
# core zoning
calibrate('potential_core.csv', 'demand.csv', 'calib_core.csv', 'patches_core.csv', 50)
# sub zoning
calibrate('potential_sub.csv', 'demand.csv', 'calib_sub.csv', 'patches_sub.csv', 50)
# all parameters
calibrate('potential_all.csv', 'demand.csv', 'calib_all.csv', 'patches_all.csv', 50)

Run Simulation

In [ ]:
def simulation(potential, demand, calib, patches, rep):
  with open(calib) as f:
    disc_factor, comp_mean, comp_range = f.readlines()[1].split(',')[:3]
  predictors = get_pred(potential)
  gs.run_command('r.futures.parallelpga', subregions=subregion, developed=dev_end,
                predictors=predictors,
                devpot_params=potential, development_pressure='devpressure_30_05_01_2011',
                n_dev_neighbourhood=30, development_pressure_approach='gravity', gamma=0.5, scaling_factor=0.1,
                demand=demand, discount_factor=disc_factor, compactness_mean=comp_mean, compactness_range=comp_range,
                patch_sizes=patches, num_neighbors=4, seed_search='probability', random_seed=42,
                output='final', output_series='step', nrpocs=nprocs, repeat=rep)

In [ ]:
simulation('potential.csv', 'demand.csv', 'calib.csv', 'patches.csv', 50)
simulation('potential_core.csv', 'demand.csv', 'calib_core.csv', 'patches_core.csv', 50)
simulation('potential_sub.csv', 'demand.csv', 'calib_sub.csv', 'patches_sub.csv', 50)
simulation('potential_all.csv', 'demand.csv', 'calib_all.csv', 'patches_all.csv', 50)

In [ ]:
#TODO look at Anna's GitHub from GIS Week GRASS computing workshop

Patch calibration if you were to calibrate by zoning district

In [ ]:
#TODO calibrate by zones to see how/if patches differ